In [ ]:
import pandas as pd
import datetime, pytz
import random

In [ ]:
def isascii_scalar(s):
    s = str(s)
    """Check if the characters in string s are in ASCII, U+0-U+7F."""
    return len(s) == len(s.encode())


def isascii(v):
    if hasattr(v, '__len__') and (not isinstance(v, str)):
        ret = []
        for i in v:
            ret.append(isascii_scalar(i))
        return ret
    else:
        return isascii_scalar(v)

In [ ]:
def translate_tz_name_scalar(s):
    return datetime.datetime.now(pytz.timezone(s)).strftime('%z')


def translate_tz_name(v):
    if hasattr(v, '__len__') and (not isinstance(v, str)):
        ret = []
        for i in v:
            ret.append(translate_tz_name_scalar(i))
        return ret
    else:
        return translate_tz_name_scalar(v)

### Filter out names with non-ascii chars

In [ ]:
users_raw = pd.read_excel("../data/users.xlsx", sheet_name="users", index_col="id")
users = users_raw.copy()

users["city_country"] = users["city"] + "_" + users["country"]
users["is_ascii"] = isascii(users['city_country'])

users = users[users["is_ascii"] == True]

### Add column with TZ as UTC Offset

In [ ]:
users["utc_offset"] = translate_tz_name(users['tz'])

### Load Workflow Def

In [ ]:
teams_df = pd.read_excel("../data/workflow.xlsx", sheet_name="teams")
teams = teams_df['Team'].tolist()

steps_df = pd.read_excel("../data/workflow.xlsx", sheet_name="steps")


### Assign users to teams (randomly)

In [ ]:
user_count = users.shape[0]
user_team_list = random.choices(teams, k=user_count)
users["team"] = user_team_list

### Assign average performance numbers to users

In [ ]:
perf=[]
for i in range(user_count):
    base_perf = 0.6
    user_perf = base_perf + round(random.uniform(0.0, 0.6) , 2)
    perf.append(user_perf)
    
users['performance'] = perf

In [ ]:
users

In [ ]:
users['name'] = users['first_name'] + ' ' + users['last_name']
users = users.drop(columns=['first_name','last_name','country','city_country','is_ascii'])


In [ ]:
users

In [ ]:
users.to_excel("../data/users_processed.xlsx",index=False)